In [ ]:
# conda install -c conda-forge bayesian-optimization
# conda install -c conda-forge xgboost

In [ ]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from lightgbm import LGBMClassifier
from bayes_opt import BayesianOptimization
# from xgboost import XGBClassifier

In [ ]:
lgb.__version__

In [ ]:
# Reading the saved dtypes Series
final_df_dtypes = \
pd.read_csv('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_fteng_G3V2_ohe_recip_20200216a_dtypes_series.csv'\
            , header=None, index_col=0, squeeze=True)
del final_df_dtypes.index.name
final_df_dtypes = final_df_dtypes.to_dict()

final_df = \
pd.read_csv('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_fteng_G3V2_ohe_recip_20200216a.csv'\
           , dtype= final_df_dtypes)

In [ ]:
final_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in final_df.columns]
final_df.info()

In [ ]:
df = final_df
del final_df
gc.collect()
df.info()

In [ ]:
df['TARGET'].value_counts()

In [ ]:
# scale_pos_weight 
282686 / 24825

---

## LGBM

In [ ]:
# del bo
# gc.collect()

In [ ]:
def lgbm_evaluate(**params):
    warnings.simplefilter('ignore')
    
    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])
    params['min_child_samples'] = int(params['min_child_samples'])
    params['max_bin'] = int(params['max_bin'])
#     params['max_drop'] = int(params['max_drop'])
    
        
    clf = LGBMClassifier(**params, 
#                          n_estimators = 2000,
#                          nthread = 2, 
                         boosting_type='goss',
#                          drop_seed = 924,
                         objective='binary',
                         scale_pos_weight = 11.387150050352467,
                         random_state = 924,
                         n_jobs = 2,
                         silent = False,
#                          importance_type (string, optional (default='split')) – 
#                          The type of feature importance to be filled into feature_importances_. 
#                          If ‘split’, result contains numbers of times the feature is used in a model. 
#                          If ‘gain’, result contains total gains of splits which use the feature.
                        )

    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]

    folds = StratifiedKFold(n_splits= 10, shuffle=True, random_state=1001)
        
    test_pred_proba = np.zeros(train_df.shape[0])
    
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        clf.fit(train_x, train_y, 
                eval_set = [(train_x, train_y), (valid_x, valid_y)], eval_metric = 'auc', 
                verbose = False, early_stopping_rounds = 200)

        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]
        
        del train_x, train_y, valid_x, valid_y
        gc.collect()

    return roc_auc_score(train_df['TARGET'], test_pred_proba)

In [ ]:
init_time = time.time()
params = { 
          'num_leaves': (7, 161), 
          'max_depth': (7, 189),
          'learning_rate': (.001, .1),
#           'n_estimators':(50, 1000),
#           'subsample_for_bin':(50000, 1000000),
#           'top_rate':(0.0 ,1.0),
          'min_split_gain': (.01, 1000),
#           'drop_rate': (0.0, 1.0),
#           'max_drop': (28, 343),
#           'skip_drop': (0.0, 1.0),
          'min_child_weight': (0.001, 1000),
          'min_child_samples': (99, 9999),
#         subsample (float, optional (default=1.)) – Subsample ratio of the training instance.
#         subsample_freq (int, optional (default=0)) – Frequence of subsample, <=0 means no enable.
#         colsample_bytree (float, optional (default=1.)) – Subsample ratio of columns when constructing each tree.
          'reg_alpha': (.00, 10.0), 
          'reg_lambda': (.00, 10.0), 
          'max_bin': (63, 511)}
bo = BayesianOptimization(lgbm_evaluate, params)
bo.maximize(init_points = 7, n_iter = 7)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

In [ ]:
params_list = bo.res
len(params_list)

In [ ]:
params_list[15]

In [ ]:
params_list[25]

In [ ]:
# 輸出參數檔案
import json
with open('../../../BDSE12-Group3/datasets/homecdt_ss_output/params_list_BayesOpt_20200216a_G3V2_ohe_recip_lgbm.txt', 'w', encoding='utf-8') as fout:
    for params in params_list:
        json.dump(params, fout) 
        fout.write("\n")

In [ ]:
# params_list[24]

In [ ]:
# no need for looping
# params_list = [bo.res[i] for i in range(10)]

In [ ]:
params_list

---

In [ ]:
# 讀取參數檔案
with open('../../../BDSE12-Group3/datasets/homecdt_ss_output/params_list_BayesOpt_20200210a.txt', 'r', encoding='utf-8') as f:
    params_list_read = list(map(json.loads,f))

In [ ]:
params_list_read[8]

In [ ]:
type(params_list_read[8])

In [ ]:
init_time = time.time()
params = {'learning_rate': (.0, .1), 
          'num_leaves': (20, 100), 
          'subsample': (.0, 1.0), 
          'max_depth': (6, 9), 
          'reg_alpha': (.00, 1.0), 
          'reg_lambda': (.00, 1.0), 
          'min_split_gain': (.0, .1),
          'min_child_weight': (20, 70)}
bo = BayesianOptimization(lgbm_evaluate, params)
bo.maximize(init_points = 5, n_iter = 10)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

---

## XGboost

In [ ]:
def xgb_evaluate(**params):
    warnings.simplefilter('ignore')
    
    params['max_depth'] = int(params['max_depth'])
        
    clf = XGBClassifier(**params, 
                        n_estimators = 2000, 
                        nthread = 5, 
                        objective= 'binary:logistic')

    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]

    folds = StratifiedKFold(n_splits= 5, shuffle=True, random_state=1001)
        
    test_pred_proba = np.zeros(train_df.shape[0])
    
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        clf.fit(train_x, train_y, 
                eval_set = [(train_x, train_y), (valid_x, valid_y)], eval_metric = 'auc', 
                verbose = False, early_stopping_rounds = 100)

        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]
        
        del train_x, train_y, valid_x, valid_y
        gc.collect()

    return roc_auc_score(train_df['TARGET'], test_pred_proba)

In [ ]:
init_time = time.time()
params = {'learning_rate': (.01, .03), 
          'subsample': (.0, 1.0), 
          'max_depth': (4, 9), 
          'reg_alpha': (.0, 1.0), 
          'reg_lambda': (.0, 1.0), 
          'scale_pos_weight': (.0, 5.0),
          'colsample_bytree': (.0, 1.0)}
bo = BayesianOptimization(xgb_evaluate, params)
bo.maximize(init_points = 5, n_iter = 5)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))